In [ ]:
import pandas
import seaborn as sns

sns.set(style="ticks")
sns.set_style("darkgrid")
# sns.set_context("paper")
sns.set_palette("colorblind")

alloc_order = ["Table", "ArrayAligned", "ArrayLocked", "ArrayAtomic", "ListLocal", "ListLocked", "Kernel"]

# Local Benchmarks

## 4K Pages

In [ ]:
pgd = pandas.concat([
    pandas.read_csv("out/bulk.csv"),
    pandas.read_csv("keep/kernel_bulk.csv"),
])
print("pages:", pgd["pages"][0])
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 2, 4, 6, 8, 10])
g.set(ylim=(0, 450))
g.set(ylabel="time in ns")
g.set(xlabel="threads")
g.set_titles("{col_name}")


In [ ]:
pgd = pandas.concat([
    pandas.read_csv("out/bulk.csv"),
    pandas.read_csv("keep/kernel_bulk.csv"),
])
pgd["get_avg"] = 10**9 / pgd["get_avg"]
pgd["put_avg"] = 10**9 / pgd["put_avg"]
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="ops")
g = sns.relplot(data=pgd, kind="line", x="x", y="ops",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 2, 4, 6, 8, 10])
g.set(ylabel="IO operations / thread / s")
g.set(xlabel="threads")
g.set_titles("{col_name}")


## 2M Pages

In [ ]:
pgd = pandas.read_csv("old/bulk_s1.csv")
print("allocs:", pgd["allocs"][0])
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[1, 2, 4, 6, 8, 10])
g.set(ylim=(0, 100))
g.set(ylabel="time in ns")
g.set(xlabel="threads")
g.set_titles("{col_name}")


# Server Benchmarks

In [ ]:
pgd = pandas.read_csv("remote/bulk_p.csv")
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
# pgd = pgd.groupby(["x", "alloc", "variable"]).min()
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order[:-1], hue_order=alloc_order[:-1])
g.set(xticks=[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96])
g.set(ylabel="time in ns")
g.set(xlabel="threads")
g.set(ylim=(0, 1400))


In [ ]:
pgd = pandas.read_csv("remote_o/bulk_p.csv")
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
# pgd = pgd.groupby(["x", "alloc", "variable"]).min()
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order[:-1], hue_order=alloc_order[:-1])
g.set(xticks=[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96])
g.set(ylabel="time in ns")
g.set(xlabel="threads")
g.set(ylim=(0, 1400))


In [ ]:
pgd = pandas.concat([
    pandas.read_csv("remote/bulk_v.csv"),
    pandas.read_csv("remote/kernel_bulk.csv")
])
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
# pgd = pgd.groupby(["x", "alloc", "variable"]).min()
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96])
g.set(ylabel="time in ns")
g.set(xlabel="threads")
g.set(ylim=(0, 1400))


In [ ]:
pgd = pandas.concat([
    pandas.read_csv("remote/bulk_v.csv"),
    pandas.read_csv("remote/kernel_bulk.csv")
])
pgd = pgd[["x", "alloc", "get_avg", "put_avg"]
          ].melt(id_vars=["x", "alloc"], value_vars=["get_avg", "put_avg"], value_name="time")
# pgd = pgd.groupby(["x", "alloc", "variable"]).min()
g = sns.relplot(data=pgd, kind="line", x="x", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=alloc_order, hue_order=alloc_order)
g.set(xticks=[4, 8, 12, 16])
g.set(ylabel="time in ns")
g.set(xlabel="threads")
g.set(ylim=(0, 300))
g.set(xlim=(0, 18))
